## RNN (Recurrent Neural Network) 으로 텍스트 분류하기

* 순환 신경망(Recurrent neural network, RNN)은 인공 신경망의 한 종류로, 유닛간의 연결이 순환적 구조를 갖는 특징을 갖고 있다. 이러한 구조는 시변적 동적 특징을 모델링 할 수 있도록 신경망 내부에 상태를 저장할 수 있게 해주므로, 순방향 신경망과 달리 내부의 메모리를 이용해 시퀀스 형태의 입력을 처리할 수 있다. 따라서 순환 인공 신경망은 필기 인식이나 음성 인식과 같이 시변적 특징을 지니는 데이터를 처리하는데 적용할 수 있다.

* 순환 신경망이라는 이름은 입력받는 신호의 길이가 한정되지 않은 동적 데이터를 처리한다는 점에서 붙여진 이름으로, 유한 임펄스 구조와 무한 임펄스 구조를 모두 일컫는다. 유한 임펄스 순환 신경망은 유향 비순환 그래프이므로 적절하게 풀어서 재구성한다면 순방향 신경망으로도 표현할 수 있지만, 무한 임펄스 순환 신경망은 유향 그래프인고로 순방향 신경망으로 표현하는 것이 불가능하다.

* 순환 신경망은 추가적인 저장공간을 가질 수 있다. 이 저장공간이 그래프의 형태를 가짐으로써 시간 지연의 기능을 하거나 피드백 루프를 가질 수도 있다. 이와 같은 저장공간을 게이트된 상태(gated state) 또는 게이트된 메모리(gated memory)라고 하며, LSTM과 게이트 순환 유닛(GRU)이 이를 응용하는 대표적인 예시이다.

* 출처 : https://ko.wikipedia.org/wiki/%EC%88%9C%ED%99%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D

<img src="https://i.imgur.com/n21TB3W.png">

* 출처 : http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

* 입력 갯수 출력 갯수에 따라서 one to many, many to one, many to many 로 나눠지지만 핵심은 타임스텝으로 이전 hidden state 의 아웃풋과 현시점의 인풋이 함께 연산
    * One to one - 가장 기본적인 모델
    * One to many - 하나의 이미지를 통해 문장으로 표현할 수 있음
    * Many to one - 영화 리뷰를 통해 긍정 또는 부정으로 감정을 분류 가능
    * Many to many - 여러 개의 단어를 입력받아 여러 개의 단어로 구성된 문장을 반환하는 번역기, 동영상의 경우 여러 개의 이미지 프레임에 대해 여러 개의 설명이나 번역 형태로 출력


RNN (Recurrent Neural Network)은 시계열 또는 자연어와 같은 시퀀스 데이터를 모델링하는 데 강력한 신경망 클래스입니다.

도식적으로, RNN 계층은 for 루프를 사용하여 시퀀스의 시간 단계를 반복하고, 지금까지 본 시간 단계에 대한 정보를 인코딩하는 내부 상태를 유지합니다.

Keras RNN API는 다음에 중점을두고 설계되었습니다.

사용 편리성: 내장 keras.layers.RNN, keras.layers.LSTM, keras.layers.GRU 레이어를 사용하여 어려운 구성 선택 없이도 반복 모델을 빠르게 구축할 수 있습니다.

사용자 정의 용이성 : 사용자 정의 동작으로 자체 RNN 셀 계층 ( for 루프의 내부 부분)을 정의하고 일반 keras.layers.RNN 계층 ( for 루프 자체)과 함께 사용할 수 있습니다. 이를 통해 최소한의 코드로 다양한 연구 아이디어를 유연한 방식으로 신속하게 프로토 타이핑 할 수 있습니다.

* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN

## 라이브러리 로드

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 미리보기 및 요약

In [ ]:
# df = pd.read_csv("seoul-120-text.csv")
df = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

In [ ]:
# 제목과 내용을 합쳐서 문서라는 파생변수를 만듭니다.
df["문서"] = df["제목"] + df["내용"]
df.head()

In [ ]:
# value_counts()로 분류별 빈도수 확인합니다.
df["분류"].value_counts()

In [ ]:
# 분류별 빈도수 값으로 불균형이 심해 전체 데이터로 예측을 하면 성능이 떨어질 수 있습니다.
# 일부 상위 분류 데이터만을 추출해 사용합니다.
df = df[df["분류"].isin(["행정", "경제", "복지"])]
# df = df[df["분류"].isin(["행정", "경제"])]

In [ ]:
# 정답(label) 값을 설정합니다.
label_name = "분류"

In [ ]:
# 독립변수(X, 문제)와 종속변수(y, 정답)를 나눕니다.
X = df["문서"]
y = df[label_name]

## label one-hot 형태로 만들기

In [ ]:
# get_dummies 를 사용하여 label 값을 one-hot 형태로 만듭니다.
y_onehot = pd.get_dummies(y)

In [ ]:
# train_test_split 으로 학습과 예측에 사용할 데이터를 나눕니다.
# 정답값은 y_onehot 으로 지정합니다.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train

## Vectorization
### Tokenizer

1. 이 클래스를 사용하면 각 텍스트를 일련의 정수(각 정수는 사전에 있는 토큰의 인덱스임) 또는 단어 수에 따라 각 토큰의 계수가 이진일 수 있는 벡터로 변환하여 텍스트 말뭉치를 벡터화할 수 있습니다.(tf-idf 기반)

2. 매개변수
- num_words
: 단어 빈도에 따라 유지할 최대 단어 수입니다. 가장 일반적인 단어 만 유지됩니다. 
-filters
: 각 요소가 텍스트에서 필터링될 문자인 문자열입니다. 기본값은 문자를 제외한 모든 구두점과 탭 및 줄 바꿈 '입니다.
- lower
: 부울. 텍스트를 소문자로 변환할지 여부입니다.
- split
: str. 단어 분할을 위한 구분 기호입니다.
- char_level
: True이면 모든 문자가 토큰으로 처리됩니다.
- oov_token
: 주어진 경우, 그것은 word_index에 추가되고 text_to_sequence 호출 중에 어휘 밖의 단어를 대체하는 데 사용됩니다.

3. 벡터화 과정
- Tokenizer 인스턴스를 생성
- fit_on_texts와 word_index를 사용하여 key value로 이루어진 딕셔너리를 생성
- texts_to_sequences를 이용하여 text 문장을 숫자로 이루어진 리스트로 변경
- 마지막으로 pad_sequences를 이용하여 리스트의 길이를 통일화


* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Tokenizer 는 데이터에 출현하는 모든 단어의 개수를 세고 빈도 수로 정렬해서 
# num_words 에 지정된 만큼만 숫자로 반환하고,나머지는 0 으로 반환합니다.
# 단어 사전의 크기를 지정해 주기 위해 vocab_size를 지정합니다.
# vocab_size는 텍스트 데이터의 전체 단어 집합의 크기입니다.

vocab_size = 1000
# oov_tok = "<oov>"
# tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer

In [ ]:
# Tokenizer 에 데이터 실제로 입력합니다.
# fit_on_texts와 word_index를 사용하여 key value로 이루어진 딕셔너리를 생성합니다.
tokenizer.fit_on_texts(X_train)

In [ ]:
# tokenizer의 word_index 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환합니다. 
# 이때, 반환 시 자동으로 소문자로 변환되어 들어가며, 느낌표나 마침표 같은 구두점은 자동으로 제거됩니다.
# 각 인덱스에 해당하는 단어가 무엇인지 확인해 봅니다.

word_to_index = tokenizer.word_index
sorted(word_to_index)[:10]

In [ ]:
# 단어별 빈도수를 확인해 봅니다.
list(tokenizer.word_counts.items())[:5]

In [ ]:
# 단어별 빈도수를 확인해 봅니다.
word_df = pd.DataFrame(tokenizer.word_counts.items(), columns = ['단어', '빈도수'])
word_df.sort_values(by="빈도수", ascending=False).T

In [ ]:
# texts_to_sequences를 이용하여 text 문장을 숫자로 이루어진 리스트로 변경합니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

## Padding

* 자연어 처리를 하다보면 각 문장(또는 문서)은 서로 길이가 다를 수 있습니다. 그런데 기계는 길이가 전부 동일한 문서들에 대해서는 하나의 행렬로 보고, 한꺼번에 묶어서 처리할 수 있습니다. 병렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업이 필요할 때가 있습니다.

* 텍스트는 신경망에 주입하기 전에 텐서로 변환되어야 합니다. 변환하는 방법에는 몇 가지가 있습니다.
    * 원-핫 인코딩(one-hot encoding)은 정수 배열을 0과 1로 이루어진 벡터로 변환합니다. 예를 들어 배열 [3, 5]을 인덱스 3과 5만 1이고 나머지는 모두 0인 10,000차원 벡터로 변환할 수 있습니다. 그다음 실수 벡터 데이터를 다룰 수 있는 층-Dense 층-을 신경망의 첫 번째 층으로 사용합니다. 이 방법은 num_words * num_reviews 크기의 행렬이 필요하기 때문에 메모리를 많이 사용합니다.
    * 다른 방법으로는, 정수 배열의 길이가 모두 같도록 패딩(padding)을 추가해 max_length * num_reviews 크기의 정수 텐서를 만듭니다. 이런 형태의 텐서를 다룰 수 있는 임베딩(embedding) 층을 신경망의 첫 번째 층으로 사용할 수 있습니다.
    * 이 튜토리얼에서는 두 번째 방식을 사용하겠습니다.
    * 텍스트의 길이가 같아야 하므로 pad_sequences 함수를 사용해 길이를 맞추겠습니다.
    
* API Document: https://www.tensorflow.org/tutorials/keras/text_classification

In [ ]:
# 독립변수를 전처리합니다. 
# 문장의 길이가 제각각인 벡터의 크기를 패딩 작업을 통해 나머지 빈 공간을 0으로 채워줍니다.
# max_length는 패딩의 기준이 됩니다.
# padding_type='post' 는 패딩을 앞(기본값)이 아닌 뒤('post')에 채웁니다.
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 500
padding_type = "post"
# padding_type = "pre"

X_train_sp = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
X_test_sp = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)


print(X_train.shape)

## Modeling
* Keras의 세 개의 내장 RNN
    * keras.layers.SimpleRNN: 이전 타임스텝의 출력이 다음 타임스텝으로 공급되는 완전히 연결된 RNN입니다.
    * keras.layers.GRU: Cho 등(2014년)에 의해 처음 제안되었습니다.
    * keras.layers.LSTM: Hochreiter 및 Schmidhuber(1997년)에 의해 처음 제안되었습니다.
* 2015년 초, LSTM 및 GRU의 재사용 가능한 오픈 소스 Python 구현이 Keras에 처음 이루어졌습니다.



* 내장 RNN의 주요 기능
    * dropout 및 recurrent_dropout 인수를 통한 반복 드롭아웃
    * go_backwards 인수를 통해 입력 시퀀스를 반대로 처리할 수 있음
    * unroll 인수를 통한 루프 언롤링(CPU에서 짧은 시퀀스를 처리할 때 속도가 크게 향상될 수 있음)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/1600px-Recurrent_neural_network_unfold.svg.png" width="400">

### simple RNN

* 층을 순서대로 쌓아 분류기(classifier)를 만듭니다.

* 첫 번째 층은 Embedding 층입니다. 이 층은 정수로 인코딩된 단어를 입력 받고 각 단어 인덱스에 해당하는 임베딩 벡터를 찾습니다. 이 벡터는 모델이 훈련되면서 학습됩니다. 이 벡터는 출력 배열에 새로운 차원으로 추가됩니다. 최종 차원은 (batch, sequence, embedding)이 됩니다.
* 기본적으로, RNN 레이어의 출력에는 샘플당 하나의 벡터가 포함됩니다. 이 벡터는 마지막 타임스텝에 해당하는 RNN 셀 출력으로, 전체 입력 시퀀스에 대한 정보를 포함합니다. 이 출력의 형상은 (batch_size, units)이고, 여기서 units는 레이어의 생성자에 전달된 units 인수에 해당합니다.
* return_sequences=True를 설정하면 RNN 레이어가 각 샘플(샘플 및 타임스텝당 하나의 벡터)에 대한 전체 출력 시퀀스도 반환할 수 있습니다. 이 출력의 형상은 (batch_size, timesteps, units)입니다.

* API Document: https://www.tensorflow.org/guide/keras/rnn
* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNNCell

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, GRU, Bidirectional, LSTM, Dropout, BatchNormalization

In [ ]:
# 하이퍼파라미터(모델링할 때 사용자가 직접 세팅해주는 값)을 설정합니다.
# vocab_size는 텍스트 데이터의 전체 단어 집합의 크기입니다.
# embedding_dim는 임베딩 할 벡터의 차원입니다.
# max_length는 패딩의 기준이 됩니다.

embedding_dim = 64

In [ ]:
# 클래스의 수는 분류될 예측값의 종류입니다.
# 정답값이 one-hot 형태로 인코딩 되어 있기 때문에 정답값의 컬럼의 수가 예측값의 종류가 됩니다.
n_class = y_train.shape[1]
n_class

In [ ]:
# # Simple RNN 레이어를 사용한 모델 (model_rnn) 정의합니다.

# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
# model.add(SimpleRNN(units=64, return_sequences=True))
# model.add(SimpleRNN(units=32))
# model.add(Dense(units=10))
# model.add(Dropout(0.1))
# model.add(Dense(n_class, activation='softmax'))

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
#     BatchNormalization(),
    Bidirectional(LSTM(32)),
#     Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(n_class, activation='softmax')
])

* https://www.tensorflow.org/guide/keras/rnn#%EC%96%91%EB%B0%A9%ED%96%A5_rnn

### Bidirectional LSTM

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Long_Short-Term_Memory.svg/1600px-Long_Short-Term_Memory.svg.png" width="400">

* LSTM(long short-term memory, 장단기 메모리)는 기울기 소실 문제를 해결하기 위해 고안된 딥 러닝 시스템이다. LSTM은 망각 게이트(forget gate)라 부르는 게이트를 추가적으로 가진다. 이 게이트를 통해 역전파시 기울기값이 급격하게 사라지거나 증가하는 문제를 방지할 수 있다. 이로써 기존의 RNN은 먼 과거의 일로부터 학습하는 것이 산술적으로 거의 불가능했지만, LSTM은 수백만 단위 시간 전의 사건으로부터도 학습할 수 있음으로서 고주파 신호뿐 아니라 저주파 신호까지도 다룰 수 있게 되었고, 이는 곧 성능의 비약적 발전을 가져왔다. 이로써 LSTM과 유사한 구조를 가진 신경망들도 많이 발표되고 있다.

* LSTM을 쌓은 뒤 CTC(영어:Connectionist temporal classification)로 이 신경망을 학습시키는 방식으로 실제 연구분야에 많이 사용되고 있다. 특히 CTC는 정렬과 인식에서 좋은 결과를 가져다주고 있다. 또한 기존의 은닉 마르코프 모형(HMM)으로는 불가능했던 문맥의존언어 학습이 가능하다는 것이 밝혀졌다.

* 양방향(Bi-directional) 순환 신경망은 길이가 정해진 데이터 순열을 통해 어떤 값이 들어오기 전과 후의 정보를 모두 학습하는 방식의 알고리즘이다. 이를 위해 순열을 왼쪽에서 오른쪽으로 읽을 RNN 하나와, 오른쪽에서 왼쪽으로 읽을 RNN 하나를 필요로 한다. 이 둘의 출력값을 조합한 뒤 지도된 결과와 비교하여 학습하는 것이다. LSTM과 병용할 때 특히 좋은 성능을 낸다는 사실이 증명되었다.


* 출처 : https://ko.wikipedia.org/wiki/%EC%88%9C%ED%99%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D

* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional

### GRU

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Gated_Recurrent_Unit%2C_base_type.svg/440px-Gated_Recurrent_Unit%2C_base_type.svg.png" width="400">

* LSTM을 변형시킨 알고리즘으로, Gradient Vanishing의 문제를 해결
* LSTM은 초기의 weight가 계속 지속적으로 업데이트되었지만, GRUs는 Update Gate와 Reset Gate를 추가하여, 과거의 정보를 어떻게 반영할 것인지 결정(GRU는 게이트가 2개, LSTM은 3개)
* Update Gate는 과거의 상태를 반영하는 Gate이며, Reset Gate는 현 시점 정보와 과거 시점 정보의 반영 여부를 결정

* 출처 : https://en.wikipedia.org/wiki/Gated_recurrent_unit

* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
# model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Bidirectional(LSTM(32)))
# model.add(Dropout(0.5))
# model.add(Dense(n_class))

## 모델 컴파일

In [ ]:
# 여러개 정답 중 하나 맞추는 문제이며, 정답값이 one-hot 형태이기 때문에
# 손실 함수는 categorical_crossentropy를 사용합니다.
model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics = ['accuracy']) 
model.summary()

## 학습

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 모델 학습을 실행합니다.
history = model.fit(X_train_sp, y_train, 
                    epochs=100, batch_size=64, callbacks=early_stop, validation_split=0.2,
                    use_multiprocessing=True)  

In [ ]:
# 모델 학습의 결과값을 데이터 프레임으로 만들어 확인합니다.
df_hist = pd.DataFrame(history.history)
df_hist.tail()

In [ ]:
# 모델 학습 결과을 그래프로 시각화합니다.
df_hist

In [ ]:
df_hist[["accuracy", "val_accuracy"]].plot()

In [ ]:
df_hist[["loss", "val_loss"]].plot()

## 예측

In [ ]:
# predict() 메서드로 모델 예측합니다.
y_pred = model.predict(X_test_sp)
y_pred[:10]

## 평가

In [ ]:
# numpy.argmax를 이용해 가장 큰 값의 인덱스들을 반환한 값(클래스 예측)을 y_predict에 할당합니다.
y_predict = np.argmax(y_pred, axis=1)
y_predict[:10]

In [ ]:
# numpy.argmax를 이용해 가장 큰 값의 인덱스들을 반환한 값(클래스 예측)을 y_test_val에 할당합니다.
y_test_val = np.argmax(y_test.values, axis=1)

In [ ]:
# 실제값과 예측값을 비교하여 맞춘 값의 평균을 확인합니다.
(y_test_val == y_predict).mean()

In [ ]:
# 모델에 설정된 손실 값 및 메트릭 값을 반환하여 평가합니다.
test_loss, test_acc = model.evaluate(X_test_sp, y_test)
test_loss, test_acc 